In [1]:
#pip install matplotlib

In [2]:
#pip install seaborn

In [3]:
#pip install scikit-learn

In [4]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [5]:
#pip install xgboost

In [6]:
pip install googlemaps

Note: you may need to restart the kernel to use updated packages.


In [7]:
# pip install holidays

In [8]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install ephem

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install meteostat 

Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, LassoCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn.ensemble import RandomForestRegressor

#one-hot encoding
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor #VIF

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

from sklearn.svm import SVC

import xgboost as xgb

#Cartesian join
from itertools import product           

In [12]:

import requests

from datetime import datetime, timedelta, date

import polyline
from haversine import haversine, Unit

Import waypoint master

In [13]:
file_path = r'C:\Users\sumat\OneDrive\Documents\NEU\ALY 6980 Capstone\waypoints_detail_df.xlsx'
waypoints_detail_df = pd.read_excel(file_path, dtype={'Pick Zip': str, 'Drop Zip': str, 'Waypoint Zip': str, 'Nearest_Station': str})

waypoints_detail_df.head()

,Customer Code,Customer Short Name,Pick Zip,Drop Zip,Sequence,Waypoint Zip,Waypoint Lat,Waypoint Lng,Duration_s_pessimistic,Location Type,%Duration_pessimistic
0,C0001,High Grade Bev - Monmouth,04103,08852,0,04103,43.689205,-70.272004,18,Start,0.000862
1,C0001,High Grade Bev - Monmouth,04103,08852,1,04103,43.688767,-70.272323,26,Checkpoint,0.001245
2,C0001,High Grade Bev - Monmouth,04103,08852,2,04103,43.688104,-70.271100,15,Checkpoint,0.000718
3,C0001,High Grade Bev - Monmouth,04103,08852,3,04103,43.687620,-70.271636,176,Checkpoint,0.008430
4,C0001,High Grade Bev - Monmouth,04103,08852,4,04103,43.680229,-70.257689,476,Checkpoint,0.022798


In [14]:
waypoints_detail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2552 entries, 0 to 2551
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer Code           2552 non-null   object 
 1   Customer Short Name     2552 non-null   object 
 2   Pick Zip                2552 non-null   object 
 3   Drop Zip                2552 non-null   object 
 4   Sequence                2552 non-null   int64  
 5   Waypoint Zip            2530 non-null   object 
 6   Waypoint Lat            2552 non-null   float64
 7   Waypoint Lng            2552 non-null   float64
 8   Duration_s_pessimistic  2552 non-null   int64  
 9   Location Type           2552 non-null   object 
 10  %Duration_pessimistic   2552 non-null   float64
dtypes: float64(3), int64(2), object(6)
memory usage: 219.4+ KB


Import future shipping

In [15]:
file_path = r'C:\Users\sumat\OneDrive\Documents\NEU\ALY 6980 Capstone\Future shipping.xlsx'
future_shipping_df = pd.read_excel(file_path, sheet_name='Future', dtype={'Pick Zip': str, 'Drop Zip': str})

# # Drop unnecessary column, remove column, drop column, delete column
future_shipping_df.drop(columns=['Random Formular', 'Static Random', 'Randome Date'], inplace=True)

# Rename the column 
future_shipping_df.rename(columns={'Static Ship Date': 'Ship_Date'}, inplace=True)

# Define mapping for truck types
mapping = {
    'TRUCK (TEMP CONTROLLED)': 'Reefer',
    'TRUCK (TEMP CONTROLLED) - MULTI STOP': 'Reefer',
    'TRUCK (DRY)': 'Dry',
    'TRUCK (DRY) - MULTISTOP': 'Dry',
    'LTL (DRY)': 'Dry'
}

# Create the new column 'truck_type'
future_shipping_df['fridge_flag'] = future_shipping_df['Mode (Load Execution)'].map(mapping)

# Define mapping for truck types
mapping = {
    'TRUCK (TEMP CONTROLLED)': 'Direct',
    'TRUCK (TEMP CONTROLLED) - MULTI STOP': 'Multistop',
    'TRUCK (DRY)': 'Direct',
    'TRUCK (DRY) - MULTISTOP': 'Multistop',
    'LTL (DRY)': 'Direct'
}

# Create the new column 'truck_type'
future_shipping_df['multistop_flag'] = future_shipping_df['Mode (Load Execution)'].map(mapping)

#Assume maximum capacity 45,000 lbs.
max_capacity = 43500
future_shipping_df['weight_capacity'] = future_shipping_df['Weight'] / max_capacity


future_shipping_df.head()

,Ship_Date,Primary Load Number,Mode (Load Execution),Pick City,Pick State,Pick Zip,Drop City,Drop State,Drop Zip,Ref: Billto,Weight,Pallets,fridge_flag,multistop_flag,weight_capacity
0,2025-02-23,T489400001,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-092324-01,26292.0,12.0,Reefer,Direct,0.604414
1,2025-02-18,T489400002,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Secaucus,NJ,07094,JVUBD0508A1,43008.0,20.0,Reefer,Direct,0.988690
2,2025-02-19,T489400003,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-030424-01,23696.0,11.0,Reefer,Direct,0.544736
3,2025-02-19,T489400004,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Carlstadt,NJ,07072,0509-ALLA,43004.0,20.0,Reefer,Direct,0.988598
4,2025-02-27,T489400005,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Secaucus,NJ,07094,JVUBD0226A,42118.0,19.0,Reefer,Direct,0.968230


In [16]:
future_shipping_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Ship_Date              83 non-null     datetime64[ns]
 1   Primary Load Number    83 non-null     object        
 2   Mode (Load Execution)  83 non-null     object        
 3   Pick City              83 non-null     object        
 4   Pick State             83 non-null     object        
 5   Pick Zip               83 non-null     object        
 6   Drop City              83 non-null     object        
 7   Drop State             83 non-null     object        
 8   Drop Zip               83 non-null     object        
 9   Ref: Billto            83 non-null     object        
 10  Weight                 83 non-null     float64       
 11  Pallets                83 non-null     float64       
 12  fridge_flag            83 non-null     object        
 13  multist

Create future shipping details

In [17]:
future_shipping_detail_df = []

# Merge air_df with county_df on the County and State columns
selected_columns = ['Ship_Date', 'Primary Load Number', 'Pick Zip', 'Drop Zip']

future_shipping_detail_df = future_shipping_df[selected_columns].merge(
    waypoints_detail_df[['Pick Zip', 'Drop Zip', 'Sequence', 'Waypoint Zip', 'Waypoint Lat', 'Waypoint Lng', 'Duration_s_pessimistic', '%Duration_pessimistic']],
    left_on=['Pick Zip', 'Drop Zip'],
    right_on=['Pick Zip', 'Drop Zip'],
    how='left'  # Use 'left' join to keep all rows in air_df
)

future_shipping_detail_df.head(5)

,Ship_Date,Primary Load Number,Pick Zip,Drop Zip,Sequence,Waypoint Zip,Waypoint Lat,Waypoint Lng,Duration_s_pessimistic,%Duration_pessimistic
0,2025-02-23,T489400001,04103,19154,0,04103,43.689205,-70.272004,18,0.000805
1,2025-02-23,T489400001,04103,19154,1,04103,43.688767,-70.272323,26,0.001162
2,2025-02-23,T489400001,04103,19154,2,04103,43.688104,-70.271100,15,0.000671
3,2025-02-23,T489400001,04103,19154,3,04103,43.687620,-70.271636,176,0.007868
4,2025-02-23,T489400001,04103,19154,4,04103,43.680229,-70.257689,476,0.021280


In [18]:
future_shipping_detail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2785 entries, 0 to 2784
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Ship_Date               2785 non-null   datetime64[ns]
 1   Primary Load Number     2785 non-null   object        
 2   Pick Zip                2785 non-null   object        
 3   Drop Zip                2785 non-null   object        
 4   Sequence                2785 non-null   int64         
 5   Waypoint Zip            2785 non-null   object        
 6   Waypoint Lat            2785 non-null   float64       
 7   Waypoint Lng            2785 non-null   float64       
 8   Duration_s_pessimistic  2785 non-null   int64         
 9   %Duration_pessimistic   2785 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 217.7+ KB


In [19]:
future_shipping_df.head(5)

,Ship_Date,Primary Load Number,Mode (Load Execution),Pick City,Pick State,Pick Zip,Drop City,Drop State,Drop Zip,Ref: Billto,Weight,Pallets,fridge_flag,multistop_flag,weight_capacity
0,2025-02-23,T489400001,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-092324-01,26292.0,12.0,Reefer,Direct,0.604414
1,2025-02-18,T489400002,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Secaucus,NJ,07094,JVUBD0508A1,43008.0,20.0,Reefer,Direct,0.988690
2,2025-02-19,T489400003,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-030424-01,23696.0,11.0,Reefer,Direct,0.544736
3,2025-02-19,T489400004,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Carlstadt,NJ,07072,0509-ALLA,43004.0,20.0,Reefer,Direct,0.988598
4,2025-02-27,T489400005,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Secaucus,NJ,07094,JVUBD0226A,42118.0,19.0,Reefer,Direct,0.968230


In [20]:
future_shipping_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Ship_Date              83 non-null     datetime64[ns]
 1   Primary Load Number    83 non-null     object        
 2   Mode (Load Execution)  83 non-null     object        
 3   Pick City              83 non-null     object        
 4   Pick State             83 non-null     object        
 5   Pick Zip               83 non-null     object        
 6   Drop City              83 non-null     object        
 7   Drop State             83 non-null     object        
 8   Drop Zip               83 non-null     object        
 9   Ref: Billto            83 non-null     object        
 10  Weight                 83 non-null     float64       
 11  Pallets                83 non-null     float64       
 12  fridge_flag            83 non-null     object        
 13  multist

In [21]:
future_shipping_df.head(5)

,Ship_Date,Primary Load Number,Mode (Load Execution),Pick City,Pick State,Pick Zip,Drop City,Drop State,Drop Zip,Ref: Billto,Weight,Pallets,fridge_flag,multistop_flag,weight_capacity
0,2025-02-23,T489400001,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-092324-01,26292.0,12.0,Reefer,Direct,0.604414
1,2025-02-18,T489400002,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Secaucus,NJ,07094,JVUBD0508A1,43008.0,20.0,Reefer,Direct,0.988690
2,2025-02-19,T489400003,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-030424-01,23696.0,11.0,Reefer,Direct,0.544736
3,2025-02-19,T489400004,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Carlstadt,NJ,07072,0509-ALLA,43004.0,20.0,Reefer,Direct,0.988598
4,2025-02-27,T489400005,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Secaucus,NJ,07094,JVUBD0226A,42118.0,19.0,Reefer,Direct,0.968230


Create waypoint master for getting weather forecast

In [22]:
forecast_waypoint_master_df = future_shipping_detail_df.groupby(
    ["Waypoint Zip", "Waypoint Lat", "Waypoint Lng"]
).first().reset_index()

# Step 2: Keep only necessary columns for weather forecasting
forecast_waypoint_master_df = forecast_waypoint_master_df[["Waypoint Zip", "Waypoint Lat", "Waypoint Lng"]]

In [23]:
forecast_waypoint_master_df.head(5)

,Waypoint Zip,Waypoint Lat,Waypoint Lng
0,01501,42.186581,-71.850080
1,01501,42.189343,-71.847530
2,01566,42.127276,-72.065459
3,01566,42.131472,-72.054144
4,01752,42.365795,-71.593429


In [24]:
forecast_waypoint_master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Waypoint Zip  182 non-null    object 
 1   Waypoint Lat  182 non-null    float64
 2   Waypoint Lng  182 non-null    float64
dtypes: float64(2), object(1)
memory usage: 4.4+ KB


In [25]:
# #test function
# test_df = forecast_waypoint_master_df[forecast_waypoint_master_df['Waypoint Zip'] == '01501']
# test_df.info()

Get weather forecast

In [26]:
def fetch_weather_forecast(latitude, longitude, forecast_days=14):
    """
    Fetches the weather forecast from the Open-Meteo API for a given latitude and longitude.

    Parameters:
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.
        forecast_days (int): Number of days to forecast (default is 14).

    Returns:
        pd.DataFrame: DataFrame containing the weather forecast for the specified location.
    """
    # Define the API URL and parameters
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "daily": ["temperature_2m_max", "temperature_2m_min", "rain_sum", "snowfall_sum", "windspeed_10m_max", "windgusts_10m_max"],
        "temperature_unit": "fahrenheit",
        "wind_speed_unit": "mph",
        "precipitation_unit": "inch",
        "timezone": "America/New_York",
        "forecast_days": forecast_days
    }

    # Send the GET request to the API
    response = requests.get(url, params=params)
    
    # Check if the response is successful
    if response.status_code == 200:
        data = response.json()  # Parse the JSON response

        # Extract daily weather data
        daily_data = data['daily']
        dates = daily_data['time']
        temp_max = daily_data['temperature_2m_max']
        temp_min = daily_data['temperature_2m_min']
        rain_sum = daily_data['rain_sum']
        snowfall_sum = daily_data['snowfall_sum']
        wind_speed_max = daily_data['windspeed_10m_max']
        wind_gusts_max = daily_data['windgusts_10m_max']

        # Create a DataFrame to store the forecast data
        forecast_df = pd.DataFrame({
            'Date': pd.to_datetime(dates),
            'High_Temp_F': temp_max,
            'Low_Temp_F': temp_min,
            'precipitation_in': rain_sum,
            'snowfall_in': snowfall_sum,
            'wind_speed_mph': wind_speed_max,
            'wind_gust_mph': wind_gusts_max
        })
        
        return forecast_df
    else:
        print(f"Error: Failed to fetch data. Status code {response.status_code}")
        return None

In [27]:
#2 minutes
# Apply the forecast function to each row
forecast_data = []

for _, row in forecast_waypoint_master_df.iterrows():
    lat, lng = row["Waypoint Lat"], row["Waypoint Lng"]
    
    # Fetch forecast for this waypoint
    forecast_df = fetch_weather_forecast(lat, lng)
    
    # Add Waypoint details to each row in forecast_df
    forecast_df["Waypoint Zip"] = row["Waypoint Zip"]
    forecast_df["Waypoint Lat"] = lat
    forecast_df["Waypoint Lng"] = lng
    
    # Append to the list
    forecast_data.append(forecast_df)

# Combine all forecast data into a single DataFrame
temp_forecast_df = pd.concat(forecast_data, ignore_index=True)

Classify temperature conditions

In [28]:
def classify_temp(row):
    if 40 <= row['High_Temp_F'] <= 60 and 40 <= row['Low_Temp_F'] <= 60:
        return {"temp_condition": "Perfect", "temp_condition_ordered": "3_Perfect"}  

    elif row['High_Temp_F'] > 65 or row['Low_Temp_F'] > 65:
        return {"temp_condition": "65+", "temp_condition_ordered": "5_65+"}

    elif 60 < row['High_Temp_F'] <= 65 or 60 < row['Low_Temp_F'] <= 65:
        return {"temp_condition": "60-65", "temp_condition_ordered": "4_60-65"}  

    elif 35 <= row['High_Temp_F'] < 40 or 35 <= row['Low_Temp_F'] < 40:
        return {"temp_condition": "35-40", "temp_condition_ordered": "2_35-40"}

    else:
        return {"temp_condition": "35-", "temp_condition_ordered": "1_35-"}

Classify wind speed conditions

In [29]:
def classify_wind_speed(row):
    if row['wind_speed_mph'] > 50:
        return 'Very Strong Wind'
    
    elif 31 <= row['wind_speed_mph'] <= 50:
        return 'Strong Wind'
    
    elif 16 <= row['wind_speed_mph'] <= 30:
        return 'Moderate Wind'
    
    elif 0 <= row['wind_speed_mph'] <= 15:  # Explicitly include 0
        return 'Light Wind'
    
    else:  # This now catches negative or other unexpected values
        return 'Unknown'  # For missing or non-standard data

Classify wind gusts conditions

In [30]:
def classify_wind_gust(row):
    if row['wind_gust_mph'] > 50:
        return 'Very Strong Wind'
    
    elif 31 <= row['wind_gust_mph'] <= 50:
        return 'Strong Wind'
    
    elif 16 <= row['wind_gust_mph'] <= 30:
        return 'Moderate Wind'
    
    elif 0 <= row['wind_gust_mph'] <= 15:  # Explicitly include 0
        return 'Light Wind'
    
    else:  # This now catches negative or other unexpected values
        return 'Unknown'  # For missing or non-standard data

In [31]:
# Apply the function to classify temperature for each row
temp_forecast_df[['temp_condition', 'temp_condition_ordered']] = temp_forecast_df.apply(classify_temp, axis=1, result_type='expand')
temp_forecast_df['wind_speed_condition'] = temp_forecast_df.apply(classify_wind_speed, axis=1)
temp_forecast_df['wind_gust_condition'] = temp_forecast_df.apply(classify_wind_gust, axis=1)

In [32]:
temp_forecast_df.head()

,Date,High_Temp_F,Low_Temp_F,precipitation_in,snowfall_in,wind_speed_mph,wind_gust_mph,Waypoint Zip,Waypoint Lat,Waypoint Lng,temp_condition,temp_condition_ordered,wind_speed_condition,wind_gust_condition
0,2025-03-24,39.7,30.6,0.610,0.193,10.0,25.9,01501,42.186581,-71.85008,35-40,2_35-40,Light Wind,Moderate Wind
1,2025-03-25,43.0,33.2,0.000,0.000,12.3,21.3,01501,42.186581,-71.85008,35-,1_35-,Light Wind,Moderate Wind
2,2025-03-26,44.2,28.8,0.043,0.496,11.7,27.7,01501,42.186581,-71.85008,35-,1_35-,Light Wind,Moderate Wind
3,2025-03-27,44.1,26.4,0.000,0.000,11.9,27.3,01501,42.186581,-71.85008,35-,1_35-,Light Wind,Moderate Wind
4,2025-03-28,46.8,32.2,0.000,0.000,13.0,29.5,01501,42.186581,-71.85008,35-,1_35-,Light Wind,Moderate Wind


In [33]:
temp_forecast_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2548 entries, 0 to 2547
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    2548 non-null   datetime64[ns]
 1   High_Temp_F             2548 non-null   float64       
 2   Low_Temp_F              2548 non-null   float64       
 3   precipitation_in        2548 non-null   float64       
 4   snowfall_in             2548 non-null   float64       
 5   wind_speed_mph          2548 non-null   float64       
 6   wind_gust_mph           2548 non-null   float64       
 7   Waypoint Zip            2548 non-null   object        
 8   Waypoint Lat            2548 non-null   float64       
 9   Waypoint Lng            2548 non-null   float64       
 10  temp_condition          2548 non-null   object        
 11  temp_condition_ordered  2548 non-null   object        
 12  wind_speed_condition    2548 non-null   object  

In [34]:
# Generate csv, #export csv
# Specify the full path and file name
file_path = r"C:\Users\sumat\OneDrive\Documents\NEU\ALY 6980 Capstone\temp_forecast_df.xlsx"

# Export DataFrame to Excel
temp_forecast_df.to_excel(file_path, index=False)

Import insteading of calling API

In [35]:
#Import
file_path = r'C:\Users\sumat\OneDrive\Documents\NEU\ALY 6980 Capstone\temp_forecast_df.xlsx'
temp_forecast_df = pd.read_excel(file_path, dtype={'Pick Zip': str, 'Drop Zip': str, 'Waypoint Zip': str})

temp_forecast_df.head()

,Date,High_Temp_F,Low_Temp_F,precipitation_in,snowfall_in,wind_speed_mph,wind_gust_mph,Waypoint Zip,Waypoint Lat,Waypoint Lng,temp_condition,temp_condition_ordered,wind_speed_condition,wind_gust_condition
0,2025-03-24,39.7,30.6,0.610,0.193,10.0,25.9,01501,42.186581,-71.85008,35-40,2_35-40,Light Wind,Moderate Wind
1,2025-03-25,43.0,33.2,0.000,0.000,12.3,21.3,01501,42.186581,-71.85008,35-,1_35-,Light Wind,Moderate Wind
2,2025-03-26,44.2,28.8,0.043,0.496,11.7,27.7,01501,42.186581,-71.85008,35-,1_35-,Light Wind,Moderate Wind
3,2025-03-27,44.1,26.4,0.000,0.000,11.9,27.3,01501,42.186581,-71.85008,35-,1_35-,Light Wind,Moderate Wind
4,2025-03-28,46.8,32.2,0.000,0.000,13.0,29.5,01501,42.186581,-71.85008,35-,1_35-,Light Wind,Moderate Wind


Shipping Details + weather

In [36]:
# Merge air_df with county_df on the County and State columns

future_shipping_detail_df = future_shipping_detail_df.merge(
    temp_forecast_df,
    left_on=['Ship_Date', 'Waypoint Lat', 'Waypoint Lng', 'Waypoint Zip'],
    right_on=['Date', 'Waypoint Lat', 'Waypoint Lng', 'Waypoint Zip'],
    how='left'  # Use 'left' join to keep all rows in air_df
)

# Drop unnecessary columns
future_shipping_detail_df.drop(columns=['Date'], inplace=True)

future_shipping_detail_df.head(5)

,Ship_Date,Primary Load Number,Pick Zip,Drop Zip,Sequence,Waypoint Zip,Waypoint Lat,Waypoint Lng,Duration_s_pessimistic,%Duration_pessimistic,High_Temp_F,Low_Temp_F,precipitation_in,snowfall_in,wind_speed_mph,wind_gust_mph,temp_condition,temp_condition_ordered,wind_speed_condition,wind_gust_condition
0,2025-02-23,T489400001,04103,19154,0,04103,43.689205,-70.272004,18,0.000805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-02-23,T489400001,04103,19154,1,04103,43.688767,-70.272323,26,0.001162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-02-23,T489400001,04103,19154,2,04103,43.688104,-70.271100,15,0.000671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-02-23,T489400001,04103,19154,3,04103,43.687620,-70.271636,176,0.007868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-02-23,T489400001,04103,19154,4,04103,43.680229,-70.257689,476,0.021280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
future_shipping_detail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2785 entries, 0 to 2784
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Ship_Date               2785 non-null   datetime64[ns]
 1   Primary Load Number     2785 non-null   object        
 2   Pick Zip                2785 non-null   object        
 3   Drop Zip                2785 non-null   object        
 4   Sequence                2785 non-null   int64         
 5   Waypoint Zip            2785 non-null   object        
 6   Waypoint Lat            2785 non-null   float64       
 7   Waypoint Lng            2785 non-null   float64       
 8   Duration_s_pessimistic  2785 non-null   int64         
 9   %Duration_pessimistic   2785 non-null   float64       
 10  High_Temp_F             1554 non-null   float64       
 11  Low_Temp_F              1554 non-null   float64       
 12  precipitation_in        1554 non-null   float64 

In [38]:
# Generate csv, #export csv
# Specify the full path and file name
file_path = r"C:\Users\sumat\OneDrive\Documents\NEU\ALY 6980 Capstone\future_shipping_detail_df.xlsx"

# Export DataFrame to Excel
future_shipping_detail_df.to_excel(file_path, index=False)

Shipping + Weather

In [39]:
# Ensure temp_condition has all expected categories
temp_conditions = ['Perfect', '65+', '60-65', '35-40', '35-']
future_shipping_detail_df['temp_condition'] = pd.Categorical(
    future_shipping_detail_df['temp_condition'], categories=temp_conditions, ordered=True
)

# Group by temp_condition and sum %Duration_pessimistic
temp_summary = (
    future_shipping_detail_df.groupby(["temp_condition", "Primary Load Number"], observed=False)["%Duration_pessimistic"]
    .sum()
    .reset_index()
)

# Pivot table to create separate columns for each temp_condition
temp_summary_pivot = temp_summary.pivot_table(
    index="Primary Load Number", columns="temp_condition", values="%Duration_pessimistic", fill_value=0
).reset_index()

# Flatten column names
temp_summary_pivot.columns.name = None  # Remove multi-index column name
temp_summary_pivot = temp_summary_pivot.rename(columns=lambda x: f"Temperature_{x}" if x != "index" else x)
temp_summary_pivot.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Temperature_Primary Load Number  83 non-null     object 
 1   Temperature_Perfect              83 non-null     float64
 2   Temperature_65+                  83 non-null     float64
 3   Temperature_60-65                83 non-null     float64
 4   Temperature_35-40                83 non-null     float64
 5   Temperature_35-                  83 non-null     float64
dtypes: float64(5), object(1)
memory usage: 4.0+ KB


In [40]:
# Merge air_df with county_df on the County and State columns
future_shipping_df = future_shipping_df.merge(
    temp_summary_pivot,
    left_on=['Primary Load Number'],
    right_on=['Temperature_Primary Load Number'],
    how='left'  # Use 'left' join to keep all rows in air_df
)

# Drop unnecessary column, remove column, drop column, delete column
future_shipping_df.drop(columns=['Temperature_Primary Load Number'], inplace=True)

future_shipping_df.head(5)

,Ship_Date,Primary Load Number,Mode (Load Execution),Pick City,Pick State,Pick Zip,Drop City,Drop State,Drop Zip,Ref: Billto,Weight,Pallets,fridge_flag,multistop_flag,weight_capacity,Temperature_Perfect,Temperature_65+,Temperature_60-65,Temperature_35-40,Temperature_35-
0,2025-02-23,T489400001,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-092324-01,26292.0,12.0,Reefer,Direct,0.604414,0.0,0.0,0.0,0.0,0.0
1,2025-02-18,T489400002,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Secaucus,NJ,07094,JVUBD0508A1,43008.0,20.0,Reefer,Direct,0.988690,0.0,0.0,0.0,0.0,0.0
2,2025-02-19,T489400003,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-030424-01,23696.0,11.0,Reefer,Direct,0.544736,0.0,0.0,0.0,0.0,0.0
3,2025-02-19,T489400004,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Carlstadt,NJ,07072,0509-ALLA,43004.0,20.0,Reefer,Direct,0.988598,0.0,0.0,0.0,0.0,0.0
4,2025-02-27,T489400005,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Secaucus,NJ,07094,JVUBD0226A,42118.0,19.0,Reefer,Direct,0.968230,0.0,0.0,0.0,0.0,0.0


In [41]:
# Generate csv, #export csv
# Specify the full path and file name
file_path = r"C:\Users\sumat\OneDrive\Documents\NEU\ALY 6980 Capstone\future_shipping_df.xlsx"

# Export DataFrame to Excel
future_shipping_df.to_excel(file_path, index=False)

14-Day Forecast - Shipping Detail 

In [42]:
future_14Day_shipping_detial_df = []

# Merge air_df with county_df on the County and State columns
selected_columns = ['Primary Load Number', 'Pick Zip', 'Drop Zip']

future_14Day_shipping_detial_df = future_shipping_df[selected_columns].merge(
    waypoints_detail_df[['Pick Zip', 'Drop Zip', 'Sequence', 'Waypoint Zip', 'Waypoint Lat', 'Waypoint Lng', 'Duration_s_pessimistic', '%Duration_pessimistic']],
    left_on=['Pick Zip', 'Drop Zip'],
    right_on=['Pick Zip', 'Drop Zip'],
    how='left'  # Use 'left' join to keep all rows in air_df
)

future_14Day_shipping_detial_df.head(5)

,Primary Load Number,Pick Zip,Drop Zip,Sequence,Waypoint Zip,Waypoint Lat,Waypoint Lng,Duration_s_pessimistic,%Duration_pessimistic
0,T489400001,04103,19154,0,04103,43.689205,-70.272004,18,0.000805
1,T489400001,04103,19154,1,04103,43.688767,-70.272323,26,0.001162
2,T489400001,04103,19154,2,04103,43.688104,-70.271100,15,0.000671
3,T489400001,04103,19154,3,04103,43.687620,-70.271636,176,0.007868
4,T489400001,04103,19154,4,04103,43.680229,-70.257689,476,0.021280


In [43]:
future_14Day_shipping_detial_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2785 entries, 0 to 2784
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Primary Load Number     2785 non-null   object 
 1   Pick Zip                2785 non-null   object 
 2   Drop Zip                2785 non-null   object 
 3   Sequence                2785 non-null   int64  
 4   Waypoint Zip            2785 non-null   object 
 5   Waypoint Lat            2785 non-null   float64
 6   Waypoint Lng            2785 non-null   float64
 7   Duration_s_pessimistic  2785 non-null   int64  
 8   %Duration_pessimistic   2785 non-null   float64
dtypes: float64(3), int64(2), object(4)
memory usage: 195.9+ KB


14-Day Forecast - Shipping Detail + Weather

In [44]:
# Merge air_df with county_df on the County and State columns

future_14Day_shipping_detial_df = future_14Day_shipping_detial_df.merge(
    temp_forecast_df,
    left_on=['Waypoint Lat', 'Waypoint Lng', 'Waypoint Zip'],
    right_on=['Waypoint Lat', 'Waypoint Lng', 'Waypoint Zip'],
    how='left'  # Use 'left' join to keep all rows in air_df
)

future_14Day_shipping_detial_df.head(5)

,Primary Load Number,Pick Zip,Drop Zip,Sequence,Waypoint Zip,Waypoint Lat,Waypoint Lng,Duration_s_pessimistic,%Duration_pessimistic,Date,High_Temp_F,Low_Temp_F,precipitation_in,snowfall_in,wind_speed_mph,wind_gust_mph,temp_condition,temp_condition_ordered,wind_speed_condition,wind_gust_condition
0,T489400001,04103,19154,0,04103,43.689205,-70.272004,18,0.000805,2025-03-24,37.0,24.5,0.268,2.398,12.6,22.8,35-40,2_35-40,Light Wind,Moderate Wind
1,T489400001,04103,19154,0,04103,43.689205,-70.272004,18,0.000805,2025-03-25,44.8,30.7,0.000,0.000,12.7,25.1,35-,1_35-,Light Wind,Moderate Wind
2,T489400001,04103,19154,0,04103,43.689205,-70.272004,18,0.000805,2025-03-26,46.2,31.7,0.000,0.000,10.2,23.9,35-,1_35-,Light Wind,Moderate Wind
3,T489400001,04103,19154,0,04103,43.689205,-70.272004,18,0.000805,2025-03-27,44.6,30.5,0.000,0.000,8.5,17.9,35-,1_35-,Light Wind,Moderate Wind
4,T489400001,04103,19154,0,04103,43.689205,-70.272004,18,0.000805,2025-03-28,46.4,32.9,0.000,0.000,14.5,28.9,35-,1_35-,Light Wind,Moderate Wind


In [45]:
future_14Day_shipping_detial_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38990 entries, 0 to 38989
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Primary Load Number     38990 non-null  object        
 1   Pick Zip                38990 non-null  object        
 2   Drop Zip                38990 non-null  object        
 3   Sequence                38990 non-null  int64         
 4   Waypoint Zip            38990 non-null  object        
 5   Waypoint Lat            38990 non-null  float64       
 6   Waypoint Lng            38990 non-null  float64       
 7   Duration_s_pessimistic  38990 non-null  int64         
 8   %Duration_pessimistic   38990 non-null  float64       
 9   Date                    38990 non-null  datetime64[ns]
 10  High_Temp_F             38990 non-null  float64       
 11  Low_Temp_F              38990 non-null  float64       
 12  precipitation_in        38990 non-null  float6

In [46]:
# Generate csv, #export csv
# Specify the full path and file name
file_path = r"C:\Users\sumat\OneDrive\Documents\NEU\ALY 6980 Capstone\future_14Day_shipping_detial_df.xlsx"

# Export DataFrame to Excel
future_14Day_shipping_detial_df.to_excel(file_path, index=False)

14-Day Forecast - Shipping

In [47]:
future_14Day_shipping_df = []

# Exclude 'Ship_Date' from future_shipping_df
future_14Day_shipping_df = future_shipping_df.drop(columns=["Ship_Date", 'Temperature_Perfect', 'Temperature_65+', 'Temperature_60-65', 'Temperature_35-40', 'Temperature_35-'])

# Group by
temporary_df = (future_14Day_shipping_detial_df.
                groupby(['Primary Load Number', 'Date'])
                .size().
                reset_index()
                .drop(columns=[0])  # Drop the count column
            )   

future_14Day_shipping_df = future_14Day_shipping_df.merge(
    temporary_df[['Primary Load Number', 'Date']],
    left_on=['Primary Load Number'],
    right_on=['Primary Load Number'],
    how='left'  # Use 'left' join to keep all rows in air_df
)

future_14Day_shipping_df.head(5)


,Primary Load Number,Mode (Load Execution),Pick City,Pick State,Pick Zip,Drop City,Drop State,Drop Zip,Ref: Billto,Weight,Pallets,fridge_flag,multistop_flag,weight_capacity,Date
0,T489400001,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-092324-01,26292.0,12.0,Reefer,Direct,0.604414,2025-03-24
1,T489400001,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-092324-01,26292.0,12.0,Reefer,Direct,0.604414,2025-03-25
2,T489400001,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-092324-01,26292.0,12.0,Reefer,Direct,0.604414,2025-03-26
3,T489400001,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-092324-01,26292.0,12.0,Reefer,Direct,0.604414,2025-03-27
4,T489400001,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-092324-01,26292.0,12.0,Reefer,Direct,0.604414,2025-03-28


14-Day Forecast - Shipping + Weather

In [48]:
# Ensure temp_condition has all expected categories
temp_conditions = ['Perfect', '65+', '60-65', '35-40', '35-']

future_14Day_shipping_detial_df['temp_condition'] = pd.Categorical(
    future_14Day_shipping_detial_df['temp_condition'], categories=temp_conditions, ordered=True
)

# Group by temp_condition and sum %Duration_pessimistic
temp_summary = (
    future_14Day_shipping_detial_df.groupby(["temp_condition", "Primary Load Number", 'Date'], observed=False)["%Duration_pessimistic"]
    .sum()
    .reset_index()
)

# Pivot table to create separate columns for each temp_condition
temp_summary_pivot = temp_summary.pivot_table(
    index=["Primary Load Number", "Date"], columns="temp_condition", values="%Duration_pessimistic", fill_value=0
).reset_index()

# Flatten column names
temp_summary_pivot.columns.name = None  # Remove multi-index column name
temp_summary_pivot = temp_summary_pivot.rename(columns=lambda x: f"Temperature_{x}" if x != "index" else x)
temp_summary_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162 entries, 0 to 1161
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Temperature_Primary Load Number  1162 non-null   object        
 1   Temperature_Date                 1162 non-null   datetime64[ns]
 2   Temperature_Perfect              1162 non-null   float64       
 3   Temperature_65+                  1162 non-null   float64       
 4   Temperature_60-65                1162 non-null   float64       
 5   Temperature_35-40                1162 non-null   float64       
 6   Temperature_35-                  1162 non-null   float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 63.7+ KB


In [49]:
# Merge air_df with county_df on the County and State columns
future_14Day_shipping_df = future_14Day_shipping_df.merge(
    temp_summary_pivot,
    left_on=['Primary Load Number', 'Date'],
    right_on=['Temperature_Primary Load Number', 'Temperature_Date'],
    how='left'  # Use 'left' join to keep all rows in air_df
)

# Drop unnecessary column, remove column, drop column, delete column
future_14Day_shipping_df.drop(columns=['Temperature_Primary Load Number', 'Temperature_Date'], inplace=True)

future_14Day_shipping_df.head(5)

,Primary Load Number,Mode (Load Execution),Pick City,Pick State,Pick Zip,Drop City,Drop State,Drop Zip,Ref: Billto,Weight,Pallets,fridge_flag,multistop_flag,weight_capacity,Date,Temperature_Perfect,Temperature_65+,Temperature_60-65,Temperature_35-40,Temperature_35-
0,T489400001,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-092324-01,26292.0,12.0,Reefer,Direct,0.604414,2025-03-24,0.177441,0.0,0.0,0.512116,0.310443
1,T489400001,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-092324-01,26292.0,12.0,Reefer,Direct,0.604414,2025-03-25,0.000000,0.0,0.0,0.500805,0.499195
2,T489400001,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-092324-01,26292.0,12.0,Reefer,Direct,0.604414,2025-03-26,0.000000,0.0,0.0,0.207931,0.792069
3,T489400001,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-092324-01,26292.0,12.0,Reefer,Direct,0.604414,2025-03-27,0.000000,0.0,0.0,0.000000,1.000000
4,T489400001,TRUCK (TEMP CONTROLLED),Portland,ME,04103,Philadelphia,PA,19154,ALA-P-092324-01,26292.0,12.0,Reefer,Direct,0.604414,2025-03-28,0.273069,0.0,0.0,0.076940,0.649991


In [50]:
future_14Day_shipping_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162 entries, 0 to 1161
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Primary Load Number    1162 non-null   object        
 1   Mode (Load Execution)  1162 non-null   object        
 2   Pick City              1162 non-null   object        
 3   Pick State             1162 non-null   object        
 4   Pick Zip               1162 non-null   object        
 5   Drop City              1162 non-null   object        
 6   Drop State             1162 non-null   object        
 7   Drop Zip               1162 non-null   object        
 8   Ref: Billto            1162 non-null   object        
 9   Weight                 1162 non-null   float64       
 10  Pallets                1162 non-null   float64       
 11  fridge_flag            1162 non-null   object        
 12  multistop_flag         1162 non-null   object        
 13  wei

In [51]:
# Generate csv, #export csv
# Specify the full path and file name
file_path = r"C:\Users\sumat\OneDrive\Documents\NEU\ALY 6980 Capstone\future_14Day_shipping_df.xlsx"

# Export DataFrame to Excel
future_14Day_shipping_df.to_excel(file_path, index=False)